# Répertoire des bilans sociaux de grandes entreprises - focus sur les salariés en situation de handicap

Objectif : extraire et mettre en forme les données de bilan social des entreprises autre qu'EDF liées aux salariés et à ceux en situation d'handicap 

Méthodologie : explorer les données mises en forme d'EDF fournies puis extraire et mettre en forme de la même manière des données d'autres entreprises